<a href="https://colab.research.google.com/github/jjablonski-it/pjatk-mhe/blob/main/MHE_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import random
import matplotlib.pyplot as plt

In [26]:
# constants
length = 100
value_range = 1000

In [27]:
data = sorted([random.randint(-value_range,value_range) for _ in range(length)])
# save data to file data.txt
with open('data.txt','w') as f:
    for i in data:
        f.write(str(i)+'\n')

In [28]:
# functions
def random_index():
  return ''.join(['%d' % random.randint(0,1) for _ in range(length)])

def index_to_set(index):
  result = []
  for i, x in enumerate(list(index)):
    if(x=='1'):
      result.append(i)
  return list(map(lambda x: data[x] ,result))

def subset_sum(index):
  return abs(sum(index_to_set(index)))

def negate_bit(index, n):
  index_list = list(index)
  bit = index_list[n]
  index_list[n] = '1' if bit=='0' else '0'
  return ''.join(index_list)

def generate_neighbours(index):
  return filter(lambda x: x != '0'*length, [negate_bit(index, x) for x in range(length)])

# def get_best_index(indexes):
#   sum_dict = {index: subset_sum(index) for index in indexes}
#   return min(sum_dict, key=sum_dict.get)

In [29]:
def tabu_search(index=random_index(), iterations_limit=100): # with going back
  tl = set() # tabu list
  l  = [index] # steps list
  best_index = index
  current_index = index
  i = 0
  while(i < iterations_limit and subset_sum(best_index)!=0 and l):
    i += 1
    # print(i, 'index', current_index, 'sum', subset_sum(current_index))
    neighbours = generate_neighbours(current_index)
    neighbour_sum_dict = {index: subset_sum(index) for index in neighbours} # {index -> sum} dict
    sorted_neighbours = map(lambda x: x[0], sorted(neighbour_sum_dict.items(), key=lambda item: item[1])) # [index] array
    valid_neighbours = list(filter(lambda x: x not in tl, sorted_neighbours))

    tl.add(current_index)
    if(valid_neighbours):
      best_local_index = valid_neighbours.pop(0)
      if(subset_sum(best_local_index) < subset_sum(best_index)):
        l.append(current_index)
        best_index = best_local_index
      current_index = best_local_index
    else:
      current_index = l.pop()
  print('iterations', i)
  return best_index

In [30]:
start_index = random_index()
# save start index to file start_index.txt
with open('start_index.txt','w') as f:
    f.write(start_index)
print('start_index', start_index)
print('tabu_search')
best_index = tabu_search(start_index, 1000)
print('start_index sum %d' % subset_sum(start_index))
print('best_index  sum %d' % subset_sum(best_index))


start_index 0101000000111110111100101100101011111000100101001011010001011010100101100011100001001011100010100011
tabu_search
iterations 56
start_index sum 42
best_index  sum 0
